In [1]:
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
seed = 0
np.random.seed(seed)
import tensorflow as tf

tf.random.set_seed(seed)
import os

#os.environ['PATH'] = os.environ['XILINX_VIVADO'] + '/bin:' + os.environ['PATH']

In [2]:
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1
import pandas as pd
#from keras.utils import np_utils

import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [3]:
df = pd.read_csv('wireless_preprocessed_approx.csv')
X = df.values[:,:-1]
Y = df.values[:,-1]

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

enc = OneHotEncoder()
Y = enc.fit_transform(Y[:, np.newaxis]).toarray()

X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.3, random_state=42)

print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(36737, 8) (15745, 8) (36737, 4) (15745, 4)


In [4]:
# Create the model
model = Sequential()
model.add(Dense(20, input_dim=8, activation='relu'))
model.add(Dense(4, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, Y_train, epochs=10, batch_size=5, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, Y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')


2024-10-19 07:47:08.507980: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2024-10-19 07:47:08.512137: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2299995000 Hz
2024-10-19 07:47:08.512308: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5d874f0 executing computations on platform Host. Devices:
2024-10-19 07:47:08.512330: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): Host, Default Version


Train on 36737 samples
Epoch 1/10
36737/36737 [==============================] - 8s 228us/sample - loss: 0.2239 - accuracy: 0.9304
Epoch 2/10
36737/36737 [==============================] - 8s 214us/sample - loss: 0.0986 - accuracy: 0.9745
Epoch 3/10
36737/36737 [==============================] - 8s 216us/sample - loss: 0.0888 - accuracy: 0.9753
Epoch 4/10
36737/36737 [==============================] - 8s 217us/sample - loss: 0.0856 - accuracy: 0.9760
Epoch 5/10
36737/36737 [==============================] - 8s 216us/sample - loss: 0.0838 - accuracy: 0.9762
Epoch 6/10
36737/36737 [==============================] - 8s 224us/sample - loss: 0.0824 - accuracy: 0.9761
Epoch 7/10
36737/36737 [==============================] - 8s 224us/sample - loss: 0.0814 - accuracy: 0.9767
Epoch 8/10
36737/36737 [==============================] - 8s 218us/sample - loss: 0.0804 - accuracy: 0.9765
Epoch 9/10
36737/36737 [==============================] - 8s 224us/sample - loss: 0.0795 - accuracy: 0.9768
Epoch

Loss: 0.08064668275091753, Accuracy: 0.9778342247009277


In [5]:
#import plotting
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
y_keras = model.predict(X_test)
print("Accuracy: {}".format(accuracy_score(np.argmax(Y_test, axis=1), np.argmax(y_keras, axis=1))))

Accuracy: 0.9778342330898698


In [6]:
y_test = enc.inverse_transform(Y_test)
y_test = [i[0] for i in y_test]

In [7]:
# Create the C file
with open('test_dataset.c', 'w') as f:
    # Write the data samples array
    f.write("float test_data[%d][%d] = {\n" % (X_test.shape[0], X_test.shape[1]))
    for row in X_test:
        f.write("    {%s},\n" % ", ".join(f"{x:.6f}f" for x in row))
    f.write("};\n\n")

    # Write the labels array
    f.write("int test_labels[%d] = {\n" % len(y_test))
    f.write("    %s\n" % ", ".join(map(str, y_test)))
    f.write("};\n")

print("Test dataset has been saved to 'test_dataset.c'")


Test dataset has been saved to 'test_dataset.c'


In [8]:
# Get the weights and biases
layer1_weights = model.layers[0].get_weights()[0].T
layer1_bias = model.layers[0].get_weights()[1]
layer2_weights = model.layers[1].get_weights()[0].T
layer2_bias = model.layers[1].get_weights()[1]

# Write weights and biases to weights.cpp
with open('weights.cpp', 'w') as f:
    f.write('#include <vector>\n\n')
    
    f.write('std::vector<std::vector<float>> layer1_weights = {\n')
    for row in layer1_weights:
        f.write('    {' + ', '.join(f'{x:.6f}f' for x in row) + '},\n')
    f.write('};\n\n')

    f.write('std::vector<float> layer1_bias = {')
    f.write(', '.join(f'{x:.6f}f' for x in layer1_bias))
    f.write('};\n\n')

    f.write('std::vector<std::vector<float>> layer2_weights = {\n')
    for row in layer2_weights:
        f.write('    {' + ', '.join(f'{x:.6f}f' for x in row) + '},\n')
    f.write('};\n\n')

    f.write('std::vector<float> layer2_bias = {')
    f.write(', '.join(f'{x:.6f}f' for x in layer2_bias))
    f.write('};\n')

print("Weights and biases have been saved to 'weights.cpp'")


Weights and biases have been saved to 'weights.cpp'


In [9]:
!g++ -std=c++11 test.cpp -o test_nn

In [10]:
!./test_nn

Accuracy: 96.0749%
